<a href="https://colab.research.google.com/github/GWANGHYUNYU/pytorch-tutorial/blob/master/08_1_xor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [101]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [102]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [103]:
def get_AND_data():
    X = [[0, 0], [0, 1], [1, 0], [1, 1]]
    Y = [[0],[0],[0],[1]]
    return X, Y

def get_OR_data():
    X = [[0, 0], [0, 1], [1, 0], [1, 1]]
    Y = [[0],[1],[1],[1]]
    return X, Y

def get_XOR_data():
    X = [[0, 0], [0, 1], [1, 0], [1, 1]]
    Y = [[0], [1], [1], [0]]
    return X, Y

In [116]:
# X, Y = get_AND_data()
# X, Y = get_OR_data()
X, Y = get_XOR_data()

In [117]:
x_train = torch.FloatTensor(X)
y_train = torch.FloatTensor(Y)

In [118]:
print(x_train.shape)
print(y_train.shape)

torch.Size([4, 2])
torch.Size([4, 1])


In [119]:
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [120]:
model = Classifier()

In [121]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = model(x_train)
    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))

Epoch    0/100 Cost: 0.743351 Accuracy 50.00%
Epoch   10/100 Cost: 0.693706 Accuracy 50.00%
Epoch   20/100 Cost: 0.693323 Accuracy 50.00%
Epoch   30/100 Cost: 0.693205 Accuracy 50.00%
Epoch   40/100 Cost: 0.693168 Accuracy 50.00%
Epoch   50/100 Cost: 0.693155 Accuracy 50.00%
Epoch   60/100 Cost: 0.693150 Accuracy 50.00%
Epoch   70/100 Cost: 0.693148 Accuracy 50.00%
Epoch   80/100 Cost: 0.693148 Accuracy 50.00%
Epoch   90/100 Cost: 0.693147 Accuracy 50.00%
Epoch  100/100 Cost: 0.693147 Accuracy 50.00%


In [35]:
# Accuracy computation
# True if hypothesis>0.5 else False
with torch.no_grad():
    hypothesis = model(x_train)
    # print(type(hypothesis))
    predicted = (hypothesis > 0.5)
    # print(predicted)
    correct_prediction = (predicted == y_train)
    # print(correct_prediction, type(correct_prediction))
    accuracy = correct_prediction.sum().item() / len(correct_prediction)
    # print(accuracy)
    print('\nHypothesis: ', hypothesis.detach().cpu().numpy(), '\nCorrect: ', predicted.detach().cpu().numpy(), '\nAccuracy: ', accuracy)


Hypothesis:  [[0.18594174]
 [0.92929256]
 [0.92633486]
 [0.9986198 ]] 
Correct:  [[False]
 [ True]
 [ True]
 [ True]] 
Accuracy:  1.0


XOR 문제를 해결하기 위한 MLP

In [86]:
X, Y = get_XOR_data()

In [87]:
print(X, Y)

[[0, 0], [0, 1], [1, 0], [1, 1]] [[0], [1], [1], [0]]


In [88]:
x_train = torch.FloatTensor(X)
y_train = torch.FloatTensor(Y)

XOR 모델을 torch.nn.Sequential 로 만드는 방법

In [66]:
# nn layers
linear1 = torch.nn.Linear(2, 2, bias=True)
linear2 = torch.nn.Linear(2, 1, bias=True)
sigmoid = torch.nn.Sigmoid()

# model
model = torch.nn.Sequential(linear1, sigmoid, linear2, sigmoid).to(device)

XOR 모델을 torch.nn.Module 로 만드는 방법

In [72]:
class XOR_Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(2, 2, bias=True)
        self.linear2 = nn.Linear(2, 1, bias=True)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.sigmoid(self.linear1(x))
        x = self.sigmoid(self.linear2(x))
        return x

In [73]:
model = XOR_Classifier()

In [74]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = model(x_train)
    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 100 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))

Epoch    0/1000 Cost: 0.705488 Accuracy 50.00%
Epoch  100/1000 Cost: 0.689583 Accuracy 50.00%
Epoch  200/1000 Cost: 0.669506 Accuracy 75.00%
Epoch  300/1000 Cost: 0.569690 Accuracy 75.00%
Epoch  400/1000 Cost: 0.462867 Accuracy 75.00%
Epoch  500/1000 Cost: 0.390877 Accuracy 75.00%
Epoch  600/1000 Cost: 0.124521 Accuracy 100.00%
Epoch  700/1000 Cost: 0.059425 Accuracy 100.00%
Epoch  800/1000 Cost: 0.037897 Accuracy 100.00%
Epoch  900/1000 Cost: 0.027592 Accuracy 100.00%
Epoch 1000/1000 Cost: 0.021615 Accuracy 100.00%


In [75]:
# Accuracy computation
# True if hypothesis>0.5 else False
with torch.no_grad():
    hypothesis = model(x_train)
    # print(type(hypothesis))
    predicted = (hypothesis > 0.5)
    # print(predicted)
    correct_prediction = (predicted == y_train)
    # print(correct_prediction, type(correct_prediction))
    accuracy = correct_prediction.sum().item() / len(correct_prediction)
    # print(accuracy)
    print('\nHypothesis: ', hypothesis.detach().cpu().numpy(), '\nCorrect: ', predicted.detach().cpu().numpy(), '\nAccuracy: ', accuracy)


Hypothesis:  [[0.02366739]
 [0.98067653]
 [0.98017913]
 [0.02252991]] 
Correct:  [[False]
 [ True]
 [ True]
 [False]] 
Accuracy:  1.0


MLP

In [92]:
X, Y = get_XOR_data()

In [93]:
x_train = torch.FloatTensor(X)
y_train = torch.FloatTensor(Y)

In [94]:
class MLP_Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(2, 5, bias=True)
        self.linear2 = nn.Linear(5, 10, bias=True)
        self.linear3 = nn.Linear(10, 5, bias=True)
        self.linear4 = nn.Linear(5, 1, bias=True)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.sigmoid(self.linear1(x))
        x = self.sigmoid(self.linear2(x))
        x = self.sigmoid(self.linear3(x))
        x = self.sigmoid(self.linear4(x))
        return x

In [95]:
model = MLP_Classifier()

In [100]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = model(x_train)
    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 100 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))

Epoch    0/1000 Cost: 0.698987 Accuracy 50.00%
Epoch  100/1000 Cost: 0.693136 Accuracy 50.00%
Epoch  200/1000 Cost: 0.693135 Accuracy 50.00%
Epoch  300/1000 Cost: 0.693135 Accuracy 50.00%
Epoch  400/1000 Cost: 0.693135 Accuracy 50.00%
Epoch  500/1000 Cost: 0.693134 Accuracy 50.00%
Epoch  600/1000 Cost: 0.693134 Accuracy 50.00%
Epoch  700/1000 Cost: 0.693134 Accuracy 50.00%
Epoch  800/1000 Cost: 0.693133 Accuracy 50.00%
Epoch  900/1000 Cost: 0.693133 Accuracy 50.00%
Epoch 1000/1000 Cost: 0.693133 Accuracy 50.00%
